# Homework 3

Austin Gill

Problems:

* 5: 2, 10abd
* 6: 5, 8
* Text addition

---

In [ ]:
# %config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
# Increase the quality of PNG images
# import matplotlib as mpl
# mpl.rcParams['figure.dpi']= 120
from scipy.integrate import solve_ivp
import seaborn as sns

sns.set()

## Problem 6.5
Real motion and measurement involves error and this problem will introduce the concepts. Assume that you have a differential drive robot with wheels that are 20cm in radius and $L$ is 12cm. Using the differential drive code (forward kinematics) from the text, develop code to simulate the robot motion when the wheel velocities are $\dot\phi_1 = 0.25 t^2$, $\dot\phi_2 = 0.5t$. The starting location is $(0, 0)$ with $\theta = 0$.

1. Plot the path of the robot on $0 \leq t \leq 5$. It should end up somewhere near $(50, 60)$.
2. Assume that you have Gaussian noise added to the omegas each time you evaluate the velocity (each time step). Test with $\mu=0$ and $\sigma=0.3$. Write the final location $(x,y)$ to a file and repeat for 100 simulations.
3. Generate a plot that includes the noise free robot path and the final locations for the simulations with noise.
4. Find the location means and 2x2 covariance matrix for this data set, and compute the eigenvalues and eigenvectors of the matrix. Find the ellipse that these generate. The major and minor axes directions are given by the eigenvectors. Show the point cloud of final locations and the ellipse in a graphic (plot the data and the ellipse).

We run the wheel speeds through a poor-man's numerical integration routine

In [ ]:
r = 20
L = 12
# Our initial position
x0 = np.array([0, 0, 0])
# The time step to use when integrating
STEP = 0.05

In [ ]:
def speeds(t):
    """Returns the (r, l) speeds at the given time"""
    return 0.25 * t**2, 0.5 * t

In [ ]:
def x_dot(w1, w2, theta):
    """x time derivative for diff. drive forward kinematics"""
    return (r / 2) * (w1 + w2) * np.cos(theta)

def y_dot(w1, w2, theta):
    """y time derivative for diff. drive forward kinematics"""
    return (r / 2) * (w1 + w2) * np.sin(theta)

def theta_dot(w1, w2):
    """theta time derivative for diff. drive forward kinematics"""
    return (r / (2 * L)) * (w1 - w2)

In [ ]:
def positions(x0):
    """Runs poor-man's integration to find robot position, given its initial position"""
    for t in np.arange(0, 5+STEP, STEP):
        # Get the wheel speeds at the given time
        w1, w2 = speeds(t)
        # Fun fact: x0 += x appends x to x0, not adds x to x0!
        x0 = x0 + np.array([STEP * x_dot(w1, w2, x0[2]), STEP * y_dot(w1, w2, x0[2]), STEP * theta_dot(w1, w2)])
        yield x0

In [ ]:
# Convert the generator to a list of 3-tuples of position points
points = list(positions(x0))
# Convert to vectors of x and y for ease of plotting.
x = np.array([point[0] for point in points])
y = np.array([point[1] for point in points])
theta = np.array([point[2] for point in points])

print('Final position: ({}, {})'.format(x[-1], y[-1]))

In [ ]:
plt.plot(x, y)
plt.title('Robot position for $0 \leq t \leq 5$')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

For fun, I wanted to try out a better integration technique than Eulers Method. For this, [`scipy.integrate.solve_ivp`](https://scipy.github.io/devdocs/generated/scipy.integrate.solve_ivp.html) is the tool to use to solve this system of ODEs. The `solve_ivp` routine takes in a function $f(t, \vec y)$, the time range $t_a \leq t \leq t_b$, and the initial condition $\vec y_0$, in order to solve the system of ordinary differential equations
$$\frac{\mathrm{d}\vec y}{\mathrm{d}t} = f(t \vec y)$$
For our problem, this is
$$\begin{pmatrix}\frac{\mathrm d x}{\mathrm d t} \\ \frac{\mathrm dy}{\mathrm dt} \\ \frac{\mathrm d\theta}{\mathrm dt}\end{pmatrix} = f(t, \langle x, y, \theta \rangle)$$
where $f$ is a function returning a vector of time-derivatives of the state variables.

So in Python, we have the following function to integrate:

In [ ]:
def ode_system(t, y):
    """Return a vector of time-derivatives at a given time and position.
    
    :param t: The current time step when integrating.
    :param y: A vector of current step estimates of the state variables during the integration.
    """
    # The wheel speeds are nicely parameterized
    w1, w2 = speeds(t)

    # I already have nifty FK functions for each of the time derivatives, so use them directly.
    xp = x_dot(w1, w2, y[2])
    yp = y_dot(w1, w2, y[2])
    tp = theta_dot(w1, w2)

    # Return a vector of derivatives of the same shape as y.
    return xp, yp, tp

In [ ]:
# Integrate the system from 0 to 5 with the initial condition (0, 0, 0)
result = solve_ivp(ode_system, (0, 5), (0, 0, 0), t_eval=np.arange(0, 5+STEP, STEP))
x_rk4, y_rk4, theta_rk4 = result.y
print('Final Position: ({}, {})'.format(x_rk4[-1], y_rk4[-1]))

Now consider the following graph of the two paths.

In [ ]:
plt.plot(x, y, label='Path using Euler\'s Method')
plt.plot(x_rk4, y_rk4, label='Path using RK45')
plt.title('Robot position for $0 \leq t \leq 5$')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.legend()
plt.show()

In [ ]:
path1 = np.array([x, y, theta])
path2 = np.array([x_rk4, y_rk4, theta_rk4])
# isclose is a boolean ndarray, so use .all() to assert that all entries are True.
print(np.isclose(path1, path2, rtol=0.1, atol=0.1).all())

Now it is quite common that even hypothetical robots (due to their spiteful nature) do not follow exact paths. They rarely follow explicit instructions, so we should repeat this process by adding random noise to the wheel speeds.

Note that if we really want the same result as adding a normal distribution with $\mu = 0$ and $\sigma = 0.3$ to the path points, we should apply some math to figure out what the $\mu$ should be if we're applying the random noise to the wheel speeds. It has something to do with a Jacobian matrix multiplied by the covariance matrix, but Dr. McGough seemed to indicate that using $\mu = 0.3$ on the wheel speeds was fine since he had already told some students to do so. So I'm going to ignore the fact that there's a little more math to do.

In [ ]:
def noisy_speeds(t):
    """Returns the (r, l) wheel speeds at the given time, with some amount of additive noise."""
    mu, sigma = 0.0, 0.3
    w1, w2 = speeds(t)
    return w1 + np.random.normal(mu, sigma), w2 + np.random.normal(mu, sigma)

Now we repeat the integration process to find the robot path

In [ ]:
def noisy_ode_system(t, y):
    """Return a vector of time-derivatives at a given time and position.
    
    :param t: The current time step when integrating.
    :param y: A vector of current step estimates of the state variables during the integration.
    """
    # The wheel speeds are nicely parameterized
    w1, w2 = noisy_speeds(t)

    # I already have nifty FK functions for each of the time derivatives, so use them directly.
    xp = x_dot(w1, w2, y[2])
    yp = y_dot(w1, w2, y[2])
    tp = theta_dot(w1, w2)

    # Return a vector of derivatives of the same shape as y.
    return xp, yp, tp

In [ ]:
# Integrate the system from 0 to 5 with the initial condition (0, 0, 0)
result = solve_ivp(noisy_ode_system, (0, 5), (0, 0, 0), t_eval=np.arange(0, 5+STEP, STEP))
n_x, n_y, n_theta = result.y
print('Final Position: ({}, {})'.format(n_x[-1], n_y[-1]))

In [ ]:
plt.plot(x_rk4, y_rk4, label='Noiseless path')
plt.plot(n_x, n_y, label='Noisy path')
plt.title('Robot position for $0 \leq t \leq 5$')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.legend()
plt.show()

Now repeat for several iterations (go get a cup of coffee)

In [ ]:
points = []
for _ in range(100):
    points.append(solve_ivp(noisy_ode_system, (0, 5), (0, 0, 0)).y)

In [ ]:
for path in points:
    x, y, theta = path
    plt.plot(x, y)

plt.title('Robot position for $0 \leq t \leq 5$')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

Or more clearly, here is a plot of the noiseless path and all of the noisy path endpoints

In [ ]:
exs = []
eys = []
for path in points:
    x, y, theta = path
    exs.append(x[-1])
    eys.append(y[-1])

plt.plot(exs, eys, 'r.', label='Noisy path endpoints')
plt.plot(x_rk4, y_rk4, 'b-', label='Noiseless path')
plt.title('Robot position for $0 \leq t \leq 5$')
plt.legend()
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

Now, being a good math student, it's probably time to do some statistical stuffs to this data. So build the covariance matrix and plot the ellipse generated by the covariance matrice's eigenvectors.

In [ ]:
cmat = np.cov(np.array([exs, eys]))
e, v = np.linalg.eigh(cmat)
angle = (180 / np.pi) * np.arctan2(v[0, 1], v[0, 0])

print(e)
print(v)
print(angle)

In [ ]:
from matplotlib.patches import Ellipse
ellipse = Ellipse((np.mean(exs), np.mean(eys)), e[0], e[1], angle, alpha=0.1)

plt.plot(exs, eys, '.', label='Noisy endpoints')
plt.plot(x_rk4[-1], y_rk4[-1], '.', label='Noiseless endpoint')
plt.legend()
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.gca().add_patch(ellipse)
plt.show()

## Problem 6.8
What are the wheel velocity formulas for a four wheel Mechanum robot, ($r=3$, $L_1=10$, $L_2=10$ all in cm) which drives in the elliptical path $\frac{(x-3)^2}{16}+\frac{(y-2)^2}{9}=1$ and always faces the center of the ellipse.

We have the parametric form of the ellipse as
$$x(t) = 3 + 4 \cos t$$
$$y(t) = 2 + 3 \sin t$$
and
$$\theta(t) = \phi(t) + \pi$$

where $\phi(t) = \arctan\frac{y}{x}$ as shown in the figure below.

![ellipse](ellipse/ellipse.svg)

However, note that both $\phi$ and therefore $\theta$ do not depend on where the center of the ellipse is, so to make the math easier for computing $\phi$, I move the center to the origin. Thus,
$$\theta(t) = \pi + \arctan\frac{4 \cos t}{3 \sin t}$$

In [ ]:
def ellipse(t):
    return 3 + 4 * np.cos(t), 2 + 3 * np.sin(t)

t = np.linspace(-np.pi - 0.1, np.pi + 0.1, 100)
x, y = ellipse(t)
# Shift the ellipse's center to the origin to avoid discontinuities in the the robot heading.
phi = np.arctan2(y - 2, x - 3)

plt.plot(t, phi)
plt.plot([-np.pi, np.pi], [-np.pi, np.pi])
plt.show()

Notice that with $-\pi \leq t \leq \pi$ we avoid the discontinuities, and more importantly that the plot is almost linear. This makes sense because, were the ellipse actually a circle, $\dot \theta$ would be constant. Since it is an ellipse, there are slight variations in the derivative as the robot goes around the "corners" of the ellipse.

Now we need to differentiate $x$, $y$, and $\theta$ with respect to $t$ in order to use the Mecanum inverse kinematics formula.

$$\dot x = -4 \sin t$$
$$\dot y = 3 \cos t$$
$$\dot \theta = \frac{12 \sec^2 t}{9 \tan^2 t + 16}$$

We can do a quick sanity check by plotting to make sure it fits with our intuitive understanding of the problem.

In [ ]:
t = np.linspace(-np.pi, np.pi, 100)
x, y = ellipse(t)
theta = np.arctan2(y, x)

xp = -4 * np.sin(t)
yp = 3 * np.cos(t)
thetap = (12 * (1 / np.cos(t))**2) / (9 * np.tan(t)**2 + 16)

plt.plot(t, xp, label=r'$\dot x$')
plt.plot(t, yp, label=r'$\dot y$')
plt.plot(t, thetap, label=r'$\dot \theta$')

plt.legend()
plt.show()

The $\dot x$ and $\dot y$ plots definitely fit my understanding of the problem, and $\dot \theta$ makes sense because there will be a decrease in the derivative while the robot goes around the "fat" corners, and a slight increase in the derivative while the robot goes around the "skinny" corners.

Now to run the inverse kinematics, which is really just an exercise in patience with $\LaTeX$.

$$\begin{bmatrix}
    \dot{\phi}_{FL} \\
    \dot{\phi}_{FR} \\
    \dot{\phi}_{BL} \\
    \dot{\phi}_{BR}
\end{bmatrix} =  \frac{1}{r}
\begin{bmatrix}
    1 & -1 & -(L_1+L_2) \\
    1 & 1 & (L_1+L_2)   \\
    1 & 1 & -(L_1+L_2)  \\
    1 & -1 & (L_1+L_2)
\end{bmatrix}
\begin{bmatrix}
    \cos(\theta) \dot{x} + \sin(\theta)\dot{y} \\
    -\sin(\theta)\dot{x} + \cos(\theta)\dot{y} \\
    \dot{\theta}
\end{bmatrix} = \frac{1}{r}
\begin{bmatrix}
    \cos(\theta) \dot{x} + \sin(\theta)\dot{y} + \sin(\theta)\dot{x} - \cos(\theta)\dot{y} -(L_1+L_2)\dot{\theta} \\
    \cos(\theta) \dot{x} + \sin(\theta)\dot{y} - \sin(\theta)\dot{x} + \cos(\theta)\dot{y} +(L_1+L_2)\dot{\theta} \\
    \cos(\theta) \dot{x} + \sin(\theta)\dot{y} - \sin(\theta)\dot{x} + \cos(\theta)\dot{y} -(L_1+L_2)\dot{\theta} \\
    \cos(\theta) \dot{x} + \sin(\theta)\dot{y} + \sin(\theta)\dot{x} - \cos(\theta)\dot{y} +(L_1+L_2)\dot{\theta}
\end{bmatrix}$$

where $r = 3$, and $L_1 = L_2 = 10$, and

$$\begin{align*}
    \dot x &= -4 \sin t \\
    \dot y &= 3 \cos t \\
    \dot \theta &= \frac{12 \sec^2 t}{9 \tan^2 t + 16} \\
    \theta(t) &= \pi + \arctan\frac{4 \cos t}{3 \sin t}
\end{align*}$$

In [ ]:
def x_dot(t):
    return -4 * np.sin(t)
def y_dot(t):
    return 3 * np.cos(t)
def theta_dot(t):
    return (12 * (1 / np.cos(t))**2) / (9 * np.tan(t)**2 + 16)
def theta(t):
    return np.pi + np.arctan2(4 * np.cos(t), 3 * np.sin(t))

In [ ]:
def mecanum_ik(t):
    r = 3
    L1 = L2 = 10
    # Avoid recomputations
    xd = x_dot(t)
    yd = y_dot(t)
    thd = theta_dot(t)
    th = theta(t)
    s = np.sin(th)
    c = np.cos(th)
    
    return (1 / r) * np.array([
        c * xd + s * yd + s * xd - c * yd - (L1 + L2) * thd,
        c * xd + s * yd - s * xd + c * yd + (L1 + L2) * thd,
        c * xd + s * yd - s * xd + c * yd - (L1 + L2) * thd,
        c * xd + s * yd + s * xd - c * yd + (L1 + L2) * thd,
    ])

In [ ]:
t = np.linspace(-np.pi, np.pi, 100)
fl, fr, bl, br = mecanum_ik(t)

In [ ]:
plt.title('Mecanum elliptic path wheel speeds')
plt.plot(t, fl, label='front left')
plt.plot(t, fr, label='front right')
plt.plot(t, bl, label='back left')
plt.plot(t, br, label='back right')
plt.xlabel('$t$')
plt.ylabel('$\dot\phi$')
plt.legend()
plt.show()

Intuitively, this plot makes sense, because as the robot moves around, each wheel needs to change direction at certain points, and due to the direction of the path, the right wheels will need to go faster than the left wheels.